In [2]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 10,
                },
                'sampling_interval': 1, # this is the logger retrieval interval
                'zero_span_check_interval': 7,
                'zero_check_duration': 3,
                'span_check_duration': 2,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
  - Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
  - Instrument current operation: ambient.
2024-05-21 17:42:55 [ne300] dtm found: 2024-05-21 17:42:54 > dtm set: 2024-05-21 17:42:54.


In [ ]:
def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['sampling_interval']).minutes.at(':00').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_logging_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response